<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création de l'évolution du CA

**Tags:** #fec #datamodel #finance #snippet #operation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataframe pour visualiser l'évolution du CA.

## Input

### Import libraries

In [1]:
import naas_data_product

✅ utils file '/home/ftp/FEC-engine/utils/datalake.ipynb' successfully loaded.


### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/BDD_ENR"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/EVOLUTION_CA"

## Model

### Récupération du dernier fichier input

In [3]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/BDD_ENR/20230524165444_FEC_BDD_ENR.csv
✅ Row fetched: 296


,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,10130000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.0,0.0,-6000.0,-inf


### Création du dataset "Evolution du CA"

In [4]:
# Creation du dataset evol_ca
dataset_evol_ca = df_input.copy()

# Filtre COMPTE_NUM = Chiffre d'Affaire (RUBRIQUE N1)
dataset_evol_ca = dataset_evol_ca[
    dataset_evol_ca["COMPTE_NUM"].str.contains(r"^70|^71|^72")
]

# Regroupement
to_group = ["ENTITY", "PERIOD", "MONTH", "MONTH_INDEX", "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
dataset_evol_ca = dataset_evol_ca.groupby(to_group, as_index=False).agg(to_agg)

dataset_evol_ca["VALUE"] = dataset_evol_ca["VALUE"].abs()


# Calcul de la somme cumulée
dataset_evol_ca = dataset_evol_ca.sort_values(
    by=["ENTITY", "PERIOD", "MONTH_INDEX"]
).reset_index(drop=True)
dataset_evol_ca["MONTH_INDEX"] = pd.to_datetime(
    dataset_evol_ca["MONTH_INDEX"], format="%m"
).dt.strftime("%m")
dataset_evol_ca["VALUE_CUM"] = dataset_evol_ca.groupby(
    ["ENTITY", "PERIOD"], as_index=True
).agg({"VALUE": "cumsum"})

# Affichage du modèle de donnée
dataset_evol_ca

KeyError: 'MONTH'

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(dataset_evol_ca, output_folder_path)